In [ ]:
# 🚀 Treinamento YOLO para Detecção de Placas

Este notebook treina um modelo YOLO personalizado para detecção de placas de carros brasileiras.

## Pré-requisitos
- Dataset de placas no formato YOLO
- GPU com CUDA (recomendado)
- Ultralytics YOLO instalado

## 1. Importações e Verificação de GPU

In [ ]:
import os
from ultralytics import YOLO
import torch
import matplotlib.pyplot as plt
from IPython.display import display, Image

In [ ]:
def print_cuda():
    """
    Verifica disponibilidade da GPU e mostra informações detalhadas
    """
    print('🔍 Verificando GPU...')
    print('GPU DISPONÍVEL:', '✅ SIM' if torch.cuda.is_available() else '❌ NÃO')
    
    if torch.cuda.is_available():
        print(f'\n📊 Informações das GPUs ({torch.cuda.device_count()} encontradas):')
        print('-' * 60)
        
        for i in range(torch.cuda.device_count()):
            props = torch.cuda.get_device_properties(i)
            allocated = torch.cuda.memory_allocated(i) / 1024**3
            cached = torch.cuda.memory_reserved(i) / 1024**3
            total = props.total_memory / 1024**3
            free = total - cached
            
            print(f"🖥️  GPU {i}: {torch.cuda.get_device_name(i)}")
            print(f"   💾 Memória Total: {total:.2f} GB")
            print(f"   🔴 Memória Alocada: {allocated:.2f} GB")
            print(f"   🟡 Memória Cached: {cached:.2f} GB")
            print(f"   🟢 Memória Livre: {free:.2f} GB")
            print(f"   🔧 Compute Capability: {props.major}.{props.minor}")
            print('-' * 60)
    else:
        print('⚠️  Treinamento será executado na CPU (muito mais lento)')

# Executa verificação
print_cuda()

## 2. Configuração do Dataset

Verifica se o dataset existe e mostra sua estrutura.

In [ ]:
# Caminho para o arquivo de configuração do dataset
dataset_config = '../../datasets/plates/data.yaml'

# Verifica se o dataset existe
if os.path.exists(dataset_config):
    print(f'✅ Dataset encontrado: {dataset_config}')
    
    # Mostra conteúdo do arquivo YAML
    with open(dataset_config, 'r') as f:
        content = f.read()
        print('\n📄 Conteúdo do data.yaml:')
        print('-' * 40)
        print(content)
        print('-' * 40)
else:
    print(f'❌ Dataset não encontrado: {dataset_config}')
    print('\n📋 Estrutura esperada:')
    print('datasets/plates/')
    print('├── data.yaml')
    print('├── images/')
    print('│   ├── train/')
    print('│   ├── val/')
    print('│   └── test/')
    print('└── labels/')
    print('    ├── train/')
    print('    ├── val/')
    print('    └── test/')

## 3. Carregamento do Modelo Base

Carrega o modelo YOLO11n pré-treinado como base.

In [ ]:
# Carrega modelo pré-treinado
print('🤖 Carregando modelo YOLO11n...')
model = YOLO("../../yolo11n.pt")

# Mostra informações do modelo
print('✅ Modelo carregado com sucesso!')
print(f'📊 Parâmetros do modelo: {sum(p.numel() for p in model.model.parameters()):,}')
print(f'💾 Tamanho aproximado: {sum(p.numel() * 4 for p in model.model.parameters()) / 1024**2:.1f} MB')

## 4. Configuração de Treinamento

Define os hiperparâmetros para um treinamento seguro e eficiente.

In [ ]:
# Configurações de treinamento
training_config = {
    # Dataset
    'data': dataset_config,
    
    # Configurações SEGURAS para evitar crash
    'workers': 4,        # Poucos workers para economizar CPU
    'device': [0],       # GPU única (mude para 'cpu' se não tiver GPU)
    'batch': 8,          # Batch pequeno para economizar VRAM
    'imgsz': 416,        # Imagem menor = menos processamento
    
    # Configurações de treinamento conservadoras
    'epochs': 100,       # Épocas suficientes para convergência
    'patience': 20,      # Para early stopping
    'optimizer': 'SGD',  # SGD usa menos memória que AdamW
    'lr0': 0.001,        # Learning rate seguro
    
    # Data augmentation reduzida (economiza processamento)
    'mosaic': 0.5,       # Reduz mosaic
    'mixup': 0.0,        # Remove mixup
    'copy_paste': 0.0,   # Remove copy-paste
    
    # Configurações de projeto
    'project': '../../runs/plate/yolo-safe',
    'name': 'safe-config',
    'exist_ok': True,
    'resume': False,
    
    # Configurações otimizadas
    'cos_lr': False,     # Remove cosine scheduler
    'warmup_epochs': 1,  # Warm-up mínimo
    'weight_decay': 0.0001,
    
    # Salvamento e monitoramento
    'save_period': 25,   # Salva checkpoints a cada 25 épocas
    'val': True,         # Validação habilitada
    'plots': True,       # Gera plots de métricas
    'verbose': True      # Output detalhado
}

# Mostra configuração
print('⚙️  Configuração de Treinamento:')
print('=' * 50)
for key, value in training_config.items():
    print(f'{key:15}: {value}')
print('=' * 50)

## 5. Treinamento do Modelo

⚠️ **ATENÇÃO:** O treinamento pode demorar várias horas dependendo do hardware disponível.

In [ ]:
# Limpa cache da GPU antes do treinamento
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print('🧹 Cache da GPU limpo')

print('\n🚀 INICIANDO TREINAMENTO...')
print('⏰ Isso pode demorar várias horas!')
print('💡 Use Ctrl+C para interromper se necessário\n')

try:
    # EXECUTA O TREINAMENTO
    results = model.train(**training_config)
    
    print('\n🎉 TREINAMENTO CONCLUÍDO COM SUCESSO!')
    print(f'📁 Resultados salvos em: {training_config["project"]}/{training_config["name"]}')
    
except KeyboardInterrupt:
    print('\n⏹️  Treinamento interrompido pelo usuário')
    
except Exception as e:
    print(f'\n❌ Erro durante o treinamento: {e}')
    print('💡 Tente reduzir o batch size ou usar CPU')

## 6. Visualização dos Resultados

Exibe gráficos e métricas do treinamento.

In [ ]:
# Caminho para os resultados
results_path = f"{training_config['project']}/{training_config['name']}"

# Lista arquivos gerados
print('📊 Arquivos gerados:')
if os.path.exists(results_path):
    for root, dirs, files in os.walk(results_path):
        level = root.replace(results_path, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            print(f'{subindent}{file}')
else:
    print('❌ Pasta de resultados não encontrada')

In [ ]:
# Exibe gráfico de resultados se existir
results_image = os.path.join(results_path, 'results.png')

if os.path.exists(results_image):
    print('📈 Gráfico de Métricas de Treinamento:')
    display(Image(results_image))
else:
    print('⚠️  Gráfico de resultados não encontrado')

In [ ]:
# Exibe matriz de confusão se existir
confusion_matrix = os.path.join(results_path, 'confusion_matrix.png')

if os.path.exists(confusion_matrix):
    print('🎯 Matriz de Confusão:')
    display(Image(confusion_matrix))
else:
    print('⚠️  Matriz de confusão não encontrada')

## 7. Teste do Modelo Treinado

Carrega o melhor modelo e testa em algumas imagens.

In [ ]:
# Caminho para o melhor modelo
best_model_path = os.path.join(results_path, 'weights', 'best.pt')

if os.path.exists(best_model_path):
    print(f'🏆 Carregando melhor modelo: {best_model_path}')
    
    # Carrega o modelo treinado
    trained_model = YOLO(best_model_path)
    
    print('✅ Modelo carregado com sucesso!')
    print(f'📁 Tamanho do arquivo: {os.path.getsize(best_model_path) / 1024**2:.1f} MB')
    
else:
    print('❌ Modelo treinado não encontrado')
    print('💡 Verifique se o treinamento foi concluído com sucesso')

In [ ]:
# Testa o modelo em imagens de exemplo (se existirem)
test_images_path = '../../images_test/'

if os.path.exists(test_images_path) and 'trained_model' in locals():
    test_images = [f for f in os.listdir(test_images_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    if test_images:
        print(f'🧪 Testando modelo em {len(test_images)} imagens...')
        
        for img_name in test_images[:3]:  # Testa apenas as 3 primeiras
            img_path = os.path.join(test_images_path, img_name)
            
            print(f'\n🖼️  Testando: {img_name}')
            
            # Executa predição
            results = trained_model(img_path, conf=0.5)
            
            # Mostra resultados
            for r in results:
                if len(r.boxes) > 0:
                    print(f'   ✅ {len(r.boxes)} placa(s) detectada(s)')
                    for i, box in enumerate(r.boxes):
                        conf = box.conf[0].item()
                        print(f'      Placa {i+1}: Confiança {conf:.2f}')
                else:
                    print('   ❌ Nenhuma placa detectada')
    else:
        print('⚠️  Nenhuma imagem de teste encontrada')
else:
    print('⚠️  Pasta de imagens de teste não encontrada ou modelo não carregado')

## 8. Informações Finais

Resumo do treinamento e próximos passos.

In [ ]:
print('🎯 RESUMO DO TREINAMENTO')
print('=' * 50)

if os.path.exists(best_model_path):
    print('✅ Status: Treinamento concluído com sucesso')
    print(f'📁 Modelo salvo em: {best_model_path}')
    print(f'📊 Resultados em: {results_path}')
    
    print('\n📋 PRÓXIMOS PASSOS:')
    print('1. 📈 Analise os gráficos de métricas acima')
    print('2. 🧪 Teste o modelo em novas imagens')
    print('3. 🔧 Ajuste hiperparâmetros se necessário')
    print('4. 🚀 Integre o modelo na sua aplicação')
    
    print('\n💡 DICAS:')
    print('• Use o arquivo best.pt para inferência')
    print('• O arquivo last.pt pode ser usado para resumir treinamento')
    print('• Monitore as métricas mAP50 e mAP50-95')
    print('• Considere mais épocas se as métricas ainda estão melhorando')
    
else:
    print('❌ Status: Treinamento não concluído ou com erro')
    print('\n🔧 SOLUÇÕES:')
    print('• Verifique se o dataset está no formato correto')
    print('• Reduza batch_size se houver erro de memória')
    print('• Use device="cpu" se não tiver GPU')
    print('• Verifique os logs de erro acima')

print('\n' + '=' * 50)